In [1]:
import subprocess
import pathlib
import os
import math

In [2]:
def make_file(exp_type, train, kernel_size, threshold, balance, model_name, 
              script_file, input_suffix='model-input', weight_decay=1e-4, 
              alpha=0.5, norm_type=0, seed=1, input_filename='exp-train'):
    assert balance in ['unbal', 'bal']
    use_all_data = True if balance == 'unbal' else False
    assert model_name in ['fb', 'dense', 'Linear1', 'Linear2', 'Conv1', 'Conv2']
    if model_name=='dense':
        model_name = 'DenseFactorBlock' 
    elif model_name=='fb':
        model_name = 'FactorBlock'
    command = ('sed "s|EXP_TYPE=\'rnaseq\'|EXP_TYPE=\'{0}\'|; '
               's/TRAIN=1/TRAIN={1}/; '
               's/KERNEL_SIZE=10/KERNEL_SIZE={2}/; '
               's/THRESHOLD=10/THRESHOLD={3}/; '
               's/USE_ALL_DATA=True/USE_ALL_DATA={4}/; '
               's/MODEL=Linear1/MODEL={5}/; '
               's|INPUT_SUFFIX=\'3rand\'|INPUT_SUFFIX=\'{6}\'|; '
               's/ALPHA=0.5/ALPHA={7}/; '
               's/NORM_TYPE=0/NORM_TYPE={8}/; '
               's/SEED=1/SEED={9}/; '
               's|WEIGHT_DECAY=1e-3|WEIGHT_DECAY={10}|" '
               '{11}.ipynb '
               '> {12}'.format(
                   exp_type, train, kernel_size, threshold, use_all_data, 
                   model_name, input_suffix, alpha, norm_type, seed, 
                   weight_decay, input_filename, script_file))
    subprocess.run(command, shell=True)
    
def submit_job(exp_type, train, kernel_size, threshold, balance, model_name, which_loss, 
               input_suffix, weight_decay=1e-4, alpha=0.5, norm_type=0, seed=1, 
               input_filename='exp-train', file_prefix='', verbose=True):
    script_path = 'notebooks/'
    ckp_path = 'checkpoints/'
    [pathlib.Path(folder).mkdir(parents=True, exist_ok=True) for folder in [script_path, ckp_path]]
    script_file_prefix = (file_prefix + '{0}_{1}_seed{2}_norm{3}-{4}-train{5}-alpha{6}-reg{7}'
                          .format(exp_type, input_suffix, seed, norm_type, model_name, train, 
                                  alpha, math.log10(weight_decay)))
    script_file = '{0}{1}.ipynb'.format(script_path, script_file_prefix)
    if not os.path.exists(script_file):
        make_file(exp_type, train, kernel_size, threshold, balance, model_name, 
                  script_file, input_suffix, weight_decay, alpha, norm_type, seed, input_filename)
        # Buggy: nbconvert will save notebook in the same path as the input file
        command = 'sbatch debug-slurm {0} {1}.ipynb'.format(script_file, script_file_prefix)
        subprocess.run(command, shell=True)
        if verbose:
            print('submitted job: {0}'.format(script_file))

In [ ]:
WHICH_LOSS = 'ce'
model_names = ['Linear1']
balances = ['unbal']
EXP_TYPES = ['rnaseq']
TRAINS = [0]
KERNEL_SIZES = [10]
THRESHOLDS = [10000]
INPUT_SUFFIXS = ['2rand2']
WEIGHT_DECAYS = [1e-3]
alphas = [0.5]
norm_types = [0]
seeds = [1,2,3,4]
for model_name in model_names:
    for balance in balances:
        for EXP_TYPE in EXP_TYPES:
            for TRAIN in TRAINS:
                for KERNEL_SIZE in KERNEL_SIZES:
                    for THRESHOLD in THRESHOLDS:
                        for INPUT_SUFFIX in INPUT_SUFFIXS:
                            for WEIGHT_DECAY in WEIGHT_DECAYS:
                                for alpha in alphas:
                                    for norm_type in norm_types:
                                        for seed in seeds:
                                            submit_job(EXP_TYPE, TRAIN, KERNEL_SIZE, THRESHOLD,
                                                       balance, model_name, WHICH_LOSS, 
                                                       INPUT_SUFFIX, WEIGHT_DECAY, alpha, 
                                                       norm_type, seed)

In [12]:
WHICH_LOSS = 'ce'
model_names = ['Linear1']
balances = ['unbal']
EXP_TYPES = ['rnaseq']
TRAINS = [0]
KERNEL_SIZES = [10]
THRESHOLDS = [10000]
INPUT_SUFFIXS = ['2rand2']
WEIGHT_DECAYS = [1e-2]
alphas = [0.5]
norm_types = [0]
seeds = [1, 2, 3, 4, 5, 6, 7, 8]
for model_name in model_names:
    for balance in balances:
        for EXP_TYPE in EXP_TYPES:
            for TRAIN in TRAINS:
                for KERNEL_SIZE in KERNEL_SIZES:
                    for THRESHOLD in THRESHOLDS:
                        for INPUT_SUFFIX in INPUT_SUFFIXS:
                            for WEIGHT_DECAY in WEIGHT_DECAYS:
                                for alpha in alphas:
                                    for norm_type in norm_types:
                                        for seed in seeds:
                                            submit_job(EXP_TYPE, TRAIN, KERNEL_SIZE, THRESHOLD,
                                                       balance, model_name, WHICH_LOSS, 
                                                       INPUT_SUFFIX, WEIGHT_DECAY, alpha, 
                                                       norm_type, seed, 
                                                       input_filename='snv-exp-train', 
                                                       file_prefix='snv_')

submitted job: notebooks/snv_rnaseq_2rand2_seed5_norm0-Linear1-train0-alpha0.5-reg-2.0.ipynb
submitted job: notebooks/snv_rnaseq_2rand2_seed6_norm0-Linear1-train0-alpha0.5-reg-2.0.ipynb
submitted job: notebooks/snv_rnaseq_2rand2_seed7_norm0-Linear1-train0-alpha0.5-reg-2.0.ipynb
submitted job: notebooks/snv_rnaseq_2rand2_seed8_norm0-Linear1-train0-alpha0.5-reg-2.0.ipynb
